In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_data, mnist_info = tfds.load('mnist', as_supervised=True, with_info=True)

mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

num_of_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_of_validation_samples = tf.cast(num_of_validation_samples, tf.int64)

num_of_test_samples = mnist_info.splits['test'].num_examples
num_of_test_samples = tf.cast(num_of_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_valdation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [3]:
BUFFER_SIZE = 10_000

In [4]:
shuffled_train_and_validation_data = scaled_train_and_valdation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_of_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_of_validation_samples)

In [5]:
BATCH_SIZE = 100
train_data = train_data.batch(batch_size=BATCH_SIZE)

In [6]:
validation_data = validation_data.batch(num_of_validation_samples)
test_data = test_data.batch(num_of_test_samples)

In [7]:
validation_inputs, validation_targets = next(iter(validation_data))

2025-11-23 20:08:53.412504: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:396] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-11-23 20:08:53.597217: W tensorflow/core/kernels/data/cache_dataset_ops.cc:917] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [8]:
inputs_size = 784
output_size = 10
hidden_layer_size = 150

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - 3ms/step - accuracy: 0.9166 - loss: 0.2854 - val_accuracy: 0.9605 - val_loss: 0.1332
Epoch 2/5
540/540 - 1s - 2ms/step - accuracy: 0.9659 - loss: 0.1119 - val_accuracy: 0.9738 - val_loss: 0.0847
Epoch 3/5
540/540 - 1s - 2ms/step - accuracy: 0.9761 - loss: 0.0753 - val_accuracy: 0.9767 - val_loss: 0.0739
Epoch 4/5
540/540 - 1s - 2ms/step - accuracy: 0.9809 - loss: 0.0588 - val_accuracy: 0.9810 - val_loss: 0.0607
Epoch 5/5
540/540 - 1s - 2ms/step - accuracy: 0.9852 - loss: 0.0462 - val_accuracy: 0.9830 - val_loss: 0.0554


In [11]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.9734 - loss: 0.0879


In [12]:
print(f"Test Loss: {test_loss}\nTest Accuracy: {test_accuracy}")

Test Loss: 0.08793295919895172
Test Accuracy: 0.9733999967575073


In [13]:
def prepare_real_digit(filepath):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_image(img, channels=1)
    
    # Resize
    img = tf.image.resize(img, [28, 28])
    
    # Normalize
    img = tf.cast(img, tf.float32) / 255.0

    # Threshold (clean it)
    img = tf.where(img > 0.3, 1.0, 0.0)

    # Add batch dimension
    img = tf.expand_dims(img, axis=0)
    return img

In [17]:
img = prepare_real_digit("number_3.jpg")
pred = model.predict(img)
print(f"Output: {tf.argmax(pred[0]).numpy()}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Output: 3
